In [1]:
%matplotlib widget
from bmcs_matmod.slide.vslide_34 import Slide34
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_matmod.slide.vslide_34 as slide_34
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from mayavi import mlab
from bmcs_matmod.slide.f_double_cap import FDoubleCap
import matplotlib.pylab as plt

# Test the return mapping for a large step beyond elastic limit

Return mapping includes also the evaluation of damage due to sliding and opening. Even though the threshold function is linear in the normal direction to the yield locus in stress space, it is nonlinear with respect to the other stress variables, i.e. $Y_\mathrm{N}, Y_\mathrm{T}$. Therefore, several iterations are required during when the step induces a high amount of damage. Following example demonstrates this for a relatively large step.
Would it make sense to visualize the threshold function in terms of f and lambda?

In [2]:
material_params = dict(
    E_T=1, gamma_T=0, K_T=-0.1, S_T=10000, c_T=1, bartau=1,
    E_N=1, S_N=10000, c_N = 1, m = 0.1, f_t=1, f_c=10, f_c0=8, eta=0
)

In [3]:
se = SlideExplorer(n_steps=2, k_max=50)
se.slide_model.trait_set(**material_params);
se.slide_model.f_lambda_recording=True # show the iteration

In [4]:
#se.trait_set(s_x_1 = 0.75, s_y_1 = 0, w_1 = -13);
#se.run()
se.trait_set(s_x_1 = 1, s_y_1 = 0, w_1 = 1);
se.run()
se.interact()

In [5]:
fig, ax = plt.subplots(1,1)
se.slide_model.lam_max = 0.8
lam_lines = np.array(se.slide_model.lam_list)
f_lines = np.array(se.slide_model.f_list)
ax.plot(lam_lines.T[:,0], f_lines.T[:,0], color='black')
ax.plot(lam_lines.T[:,1], f_lines.T[:,1], color='blue')
ax.plot(lam_lines.T[:,2], f_lines.T[:,2], color='red')
ax.plot(lam_lines.T[:,3], f_lines.T[:,3], color='yellow')
ax.plot(lam_lines.T[:,4], f_lines.T[:,4], color='orange')
ax.plot([lam_lines.T[0],lam_lines.T[-1]],[0,0], color='black', lw=0.2);
ax.set_xlim(0,0.6)
ax.set_ylim(-0.5,0.5)
ax.set_xlabel(r'$\lambda$ [-]');ax.set_ylabel(r'$f$ [MPa]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$f$ [MPa]')

# Manual return mapping to construct its visualization

In [6]:
material_params = dict(
    E_T=1, gamma_T=0, K_T=0, S_T=10000, c_T=1, bartau=1,
    E_N=1, S_N=10000, c_N = 1, m = 0.1, f_t=1, f_c=10, f_c0=8, eta=0
)

In [39]:
material_params = dict(
    E_T=1, gamma_T=0, K_T=0, S_T=10000, c_T=1, bartau=1,
    E_N=1, S_N=10000, c_N = 1, m = 0.2, f_t=2, f_c=15, f_c0=9, eta=0
)

In [40]:
s = Slide34(**material_params, k_max=50)
fdc = FDoubleCap(tau_bar=1,**material_params)

In [41]:
delta_f = 1 * s.f_t
min_sig = -s.f_c - delta_f
max_sig = s.f_t + delta_f
max_tau = s.bartau + s.m * s.f_c0 + delta_f
min_tau = -max_tau

In [42]:
min_w = min_sig / s.E_N
max_w = max_sig / s.E_N
min_s = min_tau / s.E_T
max_s = max_tau / s.E_T
s_range = np.linspace(min_s, max_s, 9)
w_range = np.linspace(min_w, max_w, 30)
s_grid, w_grid  = np.meshgrid(s_range,w_range)
s_x_n1 = np.hstack([s_grid[:,(0,-1)].flatten(), s_grid[(0,-1),1:-1].flatten()])
s_y_n1 = np.zeros_like(s_x_n1)
w_n1 = np.hstack([w_grid[:,(0,-1)].flatten(), w_grid[(0,-1),1:-1].flatten()])
#s_x_n1, w_n1

In [43]:
discr_shape = (len(w_n1),)
#discr_shape = (2,)

state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    s.state_var_shapes.items()
}

In [44]:
eps_Ema = np.c_[w_n1, s_x_n1, s_y_n1] 
#eps_Ema

In [45]:
slide = Slide34(**material_params)
discr_shape = (1,)
state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}

In [46]:
eps_aEm = np.einsum('...a->a...',eps_Ema)
select_idx = (0, 1, 2)
w_n1, s_x_n1, s_y_n1 = eps_aEm
#s_x_n1, w_n1

In [47]:
ONES = np.ones_like(s_x_n1, dtype=np.float_)
ZEROS = np.zeros_like(s_x_n1, dtype=np.float_)

In [48]:
# Transform state to Eps_k and Sig_k
Eps_n = np.array([ state[eps_name] for eps_name in slide.Eps_names], dtype=np.float_)
Eps_k = np.copy(Eps_n)
Sig_k = np.zeros_like(Eps_k)

In [49]:
Sig_k = slide.symb.get_Sig_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k)[0]
#Sig_k

<lambdifygenerated-37>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (array([[-E_N*(-2*w + 2*w_pi)*(-1/2*omega_N*select([less_equal(sig_pi, 0),True], [0,1], default=nan) + 1/2), -E_T*(1 - omega_T)*(s_pi_x - s_x), -E_T*(1 - omega_T)*(s_pi_y - s_y), K_T*z, alpha_x*gamma_T, alpha_y*gamma_T, (1/2)*E_T*(-s_pi_x + s_x)**2 + (1/2)*E_T*(-s_pi_y + s_y)**2, (1/2)*E_N*(w - w_pi)**2*select([less_equal(sig_pi, 0),True], [0,1], default=nan)]]))


In [50]:
dSig_dEps_k = slide.symb.get_dSig_dEps_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k, ZEROS, ONES)

In [51]:
H_sig_pi = slide.symb.get_H_sig_pi_(Sig_k)

In [52]:
f_k = np.array([slide.symb.get_f_(Eps_k, Sig_k, H_sig_pi)])
#f_k

In [53]:
df_dSig_k = slide.symb.get_df_dSig_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)
#df_dSig_k

In [54]:
ddf_dEps_k = slide.symb.get_ddf_dEps_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [55]:
df_dEps_k = np.einsum('ik...,ji...->jk...', df_dSig_k, dSig_dEps_k) + ddf_dEps_k

In [56]:
Phi_k = slide.get_Phi_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [57]:
dEps_dlambda_k = Phi_k

In [58]:
df_dlambda = np.einsum(
            'ki...,kj...->ij...', df_dEps_k, dEps_dlambda_k)

In [59]:
df_k = df_dlambda

In [60]:
dlam_k = -f_k / df_k
#dlam_k

In [61]:
Eps_k1 = slide.get_Eps_k1(w_n1, s_x_n1, s_y_n1, Eps_n, dlam_k, Sig_k, Eps_k)[0]
#Eps_k1.shape

In [62]:
Sig_k1 = slide.symb.get_Sig_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k1)[0]
#Sig_k1

In [63]:
dSig_dEps_k1 = slide.symb.get_dSig_dEps_(w_n1, s_x_n1, s_y_n1, Sig_k1, Eps_k1, ZEROS, ONES)

In [64]:
H_sig_pi_k1 = slide.symb.get_H_sig_pi_(Sig_k1)

In [65]:
f_k1 = np.array([slide.symb.get_f_(Eps_k1, Sig_k1, H_sig_pi_k1)])
#f_k1

In [66]:
sig_ma_k = np.c_[Sig_k[0], Sig_k[1], f_k[0]].T
#sig_ma_k

In [67]:
sig_ma_k1 = np.hstack([Sig_k1[:2,:].T, f_k1[0][:,np.newaxis]]).T
#sig_ma_k1

In [68]:
vec_ma_k1 = sig_ma_k1 - sig_ma_k
#vec_ma_k1

In [69]:
X_a, Y_a = np.mgrid[min_sig:max_sig:210j, min_tau:max_tau:210j]
Z_a = fdc.symb.get_f_solved(X_a, Y_a)
Z_0 = np.zeros_like(Z_a)

In [71]:
mlab.options.backend = 'envisage'
mlab.figure(bgcolor=(1,1,1),fgcolor=(0,0,0))
surf = mlab.surf(X_a, Y_a, Z_a, colormap='summer')
surf.actor.property.opacity = 0.7829
mlab.surf(X_a, Y_a, Z_0, color=(.8,.8,.8))
vectors = mlab.quiver3d(sig_ma_k[0],sig_ma_k[1],sig_ma_k[2],
             vec_ma_k1[0],vec_ma_k1[1],vec_ma_k1[2])
vectors.glyph.glyph_source.glyph_source.filled = True
vectors.glyph.color_mode = 'no_coloring'
vectors.actor.property.ambient_color = (1.0, 0.6666666666666666, 0.0)
vectors.actor.property.diffuse_color = (1.0, 0.6666666666666666, 0.0)
vectors.actor.property.emissive_factor = np.array([1., 1., 1.])
vectors.actor.property.specular_color = (1.0, 0.6666666666666666, 0.0)
#vectors.actor.property.color = (1.0, 0.6666666666666666, 0.0)
vectors.actor.property.color = (0.0, 0.3333333333333333, 1.0)
vectors.glyph.glyph_source.glyph_source = vectors.glyph.glyph_source.glyph_dict['cone_source']
vectors.glyph.glyph.range = np.array([0.        , 3.41082282])
vectors.glyph.glyph_source.glyph_source.radius = 0.03
vectors.glyph.glyph.scale_factor = 4
xx = np.linspace(min_sig,max_sig,2)
yy = np.linspace(min_tau,max_tau,2)
zz = np.linspace(-1,1,2)
xy = xz = yx = yz = zx = zy = np.zeros_like(xx)
mlab.plot3d(yx,yy,yz,line_width=0.01,tube_radius=0.01)
mlab.plot3d(zx,zy,zz,line_width=0.01,tube_radius=0.01)
mlab.plot3d(xx,xy,xz,line_width=0.01,tube_radius=0.01)
mlab.show()